In [ ]:
import tkinter as tk
from tkinter import messagebox
import numpy as np
import networkx as nx
import random


root = tk.Tk()
root.geometry("1200x900")  # 初始化視窗大小
root.title("兒童新樂園遊玩路徑演算法")
root.configure(bg="#FFCCCB")  # 背景顏色是淡粉紅色

# 初始化移動距離矩陣，無邊連接處設為無限大 (即無法直接到達)
move_times = np.full((13, 13), np.inf)  # 13x13的矩陣，初始值設為無限大
for i in range(13):
    move_times[i][i] = 0  # 設定自己到自己的移動時間為0

# 設置新的移動距離數據，前兩項是設施編號，最後一項是移動時間
move_data = [
    (1, 2, 3), (1, 9, 2), (1, 13, 2), (2, 5, 2), (2, 9, 2),
    (3, 5, 1), (3, 10, 2), (4, 9, 3), (4, 13, 1), (5, 6, 2),
    (6, 7, 1), (6, 10, 1), (6, 11, 2), (7, 8, 1), (7, 12, 2),
    (7, 13, 6), (8, 11, 1), (9, 13, 1)
]

# 根據移動數據更新矩陣值
for x, y, d in move_data:
    move_times[x-1][y-1] = d
    move_times[y-1][x-1] = d

# 設施名稱
facility_names = [
    "海洋總動員", "摩天輪", "銀河號", "飛天神奇號", "宇宙迴旋",
    "星空小飛碟", "轉轉咖啡杯", "巡弋飛椅", "尋寶船", "魔法星際飛車",
    "小飛龍", "幸福碰碰車", "叢林吼吼樹屋"
]

# 設施進行時間 (分鐘) 和遊玩一輪人數容量
durations = [3, 10, 4, 3, 3, 2, 3, 3, 4, 2, 3, 3, 2]
capacities = [24, 40, 25, 12, 16, 30, 32, 32, 30, 12, 32, 20, 16]

# 生成隨機進場人數
def generate_random_visitors():
    return random.randint(300, 1000)

# 設定進場人數並顯示
visitors = generate_random_visitors()
visitors_text = f"目前進場人數: {visitors} 人"

# 隱藏設施名稱和人數狀況(按按鈕之後)
def hide_facility_info():
    facility_names_label.pack_forget()
    visitor_distribution_label.pack_forget()

# 計算最短路徑
def calculate_shortest_path(start=None):
    hide_facility_info()  # 隱藏設施名稱和人數狀況

    G = nx.from_numpy_array(move_times)  # 根據移動時間矩陣創建圖
    path = nx.approximation.traveling_salesman_problem(G, cycle=True)  # 計算旅行推銷員問題的近似解

    if start is not None and start in range(1, 14):
        start -= 1
        start_index = path.index(start)
        path = path[start_index:] + path[:start_index] + [start]  # 調整起點位置

    # 確保相鄰的設施不重複
    final_path = [path[0]]
    for i in range(1, len(path) - 1):
        if path[i] != final_path[-1]:
            final_path.append(path[i])
    final_path.append(path[-1])

    length = sum(move_times[final_path[i]][final_path[i + 1]] for i in range(len(final_path) - 1))  # 計算總移動時間
    path_text = " -> ".join(str(p + 1) for p in final_path)
    result_text.set(f"最短路徑: {path_text} \n移動總時間: {length:.2f} 分鐘")

    # 隨機生成設施的人數分布並顯示
    visitor_distribution = [0] * 13
    for _ in range(visitors):
        chosen_facility = random.choices(range(13), weights=capacities, k=1)[0]
        visitor_distribution[chosen_facility] += 1

    facility_status = "各設施目前的人數狀況:\n"
    for i in range(13):
        facility_status += f"{i + 1}.{facility_names[i]}: {visitor_distribution[i]} 人\n"
    visitor_distribution_var.set(facility_status)
    visitor_distribution_label.pack(pady=20)

# 模擬安排遊客並計算最佳路徑
def calculate_best_path(start=None):
    hide_facility_info()  # 隱藏設施信息

    # 隨機生成設施的人數分布並顯示
    visitor_distribution = [0] * 13
    for _ in range(visitors):
        chosen_facility = random.choices(range(13), weights=capacities, k=1)[0]
        visitor_distribution[chosen_facility] += 1

    # 自定義最佳路徑計算邏輯，考慮移動時間、設施進行時間和消化人數
    def path_cost(path):
        total_cost = 0
        for i in range(len(path) - 1):
            total_cost += move_times[path[i]][path[i + 1]]
        return total_cost

    G = nx.from_numpy_array(move_times)  # 根據移動時間矩陣創建圖
    path = nx.approximation.traveling_salesman_problem(G, cycle=True)  # 計算旅行商問題的近似解

    if start is not None and start in range(1, 14):
        start -= 1
        start_index = path.index(start)
        path = path[start_index:] + path[:start_index] + [start]  # 調整起點位置

    # 確保相鄰的設施不重複
    final_path = [path[0]]
    for i in range(1, len(path) - 1):
        if path[i] != final_path[-1]:
            final_path.append(path[i])
    final_path.append(path[-1])

    best_cost = path_cost(final_path)  # 計算最佳路徑的成本
    best_path = final_path

    total_move_time = best_cost
    total_play_time = sum(durations[facility] for facility in best_path)  # 計算總遊玩時間
    total_queue_time = sum(visitor_distribution[i] / capacities[i] for i in range(len(visitor_distribution))) * 10  # 假設每位遊客平均排隊時間為10分鐘
    total_time = total_move_time + total_play_time + total_queue_time

    path_text = " -> ".join(str(p + 1) for p in best_path)
    result_text.set(
        f"最佳路徑: {path_text} \n總移動時間: {total_move_time:.2f} 分鐘\n總遊玩時間: {total_play_time:.2f} 分鐘\n總排隊時間: {total_queue_time:.2f} 分鐘\n總時間: {total_time:.2f} 分鐘")

    facility_status = "各設施目前的人數狀況:\n"
    for i in range(13):
        facility_status += f"{i + 1}.{facility_names[i]}: {visitor_distribution[i]} 人\n"
    visitor_distribution_var.set(facility_status)
    visitor_distribution_label.pack(pady=20)

# 界面元素
title_label = tk.Label(root, text="兒童新樂園遊玩路徑演算法", font=("Microsoft JhengHei", 40, "bold"), bg="#FFCCCB")
title_label.pack(pady=10)

visitor_label = tk.Label(root, text=visitors_text, font=("Microsoft JhengHei", 18), bg="#FFCCCB")
visitor_label.pack(pady=10)

start_label = tk.Label(root, text="從哪個設施開始遊玩 (1-13, 非必填):", font=("Microsoft JhengHei", 18), bg="#FFCCCB")
start_label.pack(pady=10)

start_entry = tk.Entry(root, font=("Times New Roman", 18))
start_entry.pack(pady=5)

# 計算最短路徑按鈕
calculate_shortest_button = tk.Button(root, text="計算最短路徑", font=("Microsoft JhengHei", 18, "bold"), fg="white", bg="#FF69B4",
                                      command=lambda: calculate_shortest_path(int(start_entry.get()) if start_entry.get().isdigit() else None))
calculate_shortest_button.pack(pady=5)

# 計算最佳路徑按鈕
calculate_best_button = tk.Button(root, text="計算最佳路徑", font=("Microsoft JhengHei", 18, "bold"), fg="white", bg="#FF69B4",
                                  command=lambda: calculate_best_path(int(start_entry.get()) if start_entry.get().isdigit() else None))
calculate_best_button.pack(pady=5)

result_text = tk.StringVar()
result_label = tk.Label(root, textvariable=result_text, font=("Microsoft JhengHei", 14), bg="#FFCCCB")
result_label.pack(pady=20)

facility_names_label = tk.Label(root, text="各個遊樂設施:", font=("Microsoft JhengHei", 14), bg="#FFCCCB")
facility_names_label.pack(pady=5)

facility_info = "\n".join([f"{i + 1}.{name}" for i, name in enumerate(facility_names)])
facility_names_var = tk.StringVar(value=facility_info)
facility_names_label = tk.Label(root, textvariable=facility_names_var, font=("Microsoft JhengHei", 14), bg="#FFCCCB")
facility_names_label.pack(pady=10)

visitor_distribution_var = tk.StringVar()
visitor_distribution_label = tk.Label(root, textvariable=visitor_distribution_var, font=("Microsoft JhengHei", 14), bg="#FFCCCB")
visitor_distribution_label.pack(pady=20)

# 運行應用程序
root.mainloop()
